the function func_run contains the main body of code which reads in data, solves Geo-I, and p mpc-H. it takes as an input the number of user.


In [1]:
import pandas as pd
import numpy as np
import math 
from casadi import *
from pylab import *
import os 
from datetime import datetime, timedelta
import pymap3d # python3 -m pip install pymap3d
import random
import pickle
from scipy.special import lambertw
from scipy import stats
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column
from bokeh.models import Range1d
output_notebook() 



Loading BokehJS ...

In [2]:
nbuf = 30

# Functions to use

In [3]:
def system_update(system,x,y,b):
    dict = {'X': x, 'Y': y, 'N': b}
    system = system.append(dict, ignore_index = True).drop(index=[0]).reset_index(drop=True)
    return system

def privacy(system):
    n = system['N'].sum()
    if n == 0:
        barx = 0
        bary = 0
        priv = 0
    else:
        barx = np.dot(system['X'], system['N'])/n
        bary = np.dot(system['Y'], system['N'])/n
        #priv = np.sqrt(np.dot((system['X'] - barx)**2 + (system['Y'] - bary)**2, system['N'])/n) 
        priv = np.dot(np.sqrt((system['X'] - barx)**2 + (system['Y'] - bary)**2), system['N'])/n 
    return barx, bary, priv

def utility(x,y):
    return np.sqrt(x**2 + y**2)

def reset_system(system):
    n = len(system)
    X = np.array([0]*n)
    Y = np.array([0]*n)
    N = np.array([0]*n)

    system['X'] = X
    system['Y'] = Y
    system['N'] = N
    return system



def GeoI_LPPM(x,y,epsilon):
    theta_GeoI = random.uniform(0,2*math.pi)
    r_GeoI = -1/epsilon*(np.real(lambertw((random.random()-1)/math.exp(1),k=-1))+1);
    x_ctrl = x + r_GeoI*math.cos(theta_GeoI)
    y_ctrl = y + r_GeoI*math.sin(theta_GeoI)
    return x_ctrl, y_ctrl, r_GeoI 
    

## Functions FLAIR

In [4]:
def prediction(horizon, coef, last_point, time_gap):
    """Returns an array of size horizon for which index i has value = last_point + (time_gap + 30*i)*(coef)

    Args:
        last_point (float): the last transmitted position (either x-wise or y -wise)
        horizon (int)
        coef (float): the linear coefficient of the current model 

    Returns:
        Xf: the array of predicted positions
    """
    #print(coef, time_gap, last_point)
    return(np.array([last_point + (time_gap + 30*i)*(coef) for i in range(horizon)]))

def FLAIR_insert(x0, x1, x, t0, t1, t, A0, A_min, A_max, epsilon):
    """Inserts a new sample (x,t) into the previous model if possible. Creates a new model otherwise.

    Args:
        x0 (_type_): first sample of the last model
        x1 (_type_): last sample of the last model
        x (_type_): sample to be inserted
        t0 (_type_): first time..
        t1 (_type_): _description_
        t (_type_): _description_
        A0 (_type_): linear coef of the current model
        A_min (_type_): _description_
        A_max (_type_): _description_
        S (_type_): stores x
        A (_type_): stores linear coefs
        T (_type_): stores time

    Returns:
        _type_: _description_
    """
    x_delta, t_delta = x -x0, t - t0
    
    A_t = x_delta/(t_delta+1e-12)
    if A_min <= A_t and A_t <= A_max:
        
        A0 = A_t
        A_min = max(A_min, (x_delta-epsilon)/(t_delta + 1e-12))
        A_max = min(A_max, (x_delta+epsilon)/(t_delta + 1e-12))
    else: 
        t0 = t
        x0, t0 = x1, t1
        x_delta, t_delta = x -x0, t -t0
        A0 = x_delta/(t_delta+1e-12)
        A_min = (x_delta-epsilon)/(t_delta + 1e-12)
        A_max = (x_delta+epsilon)/(t_delta + 1e-12)
    x1, t1 = x, t 
    return (x0, t0, x1, t1, A0, A_min, A_max)





def predict_xf(current_FLAIR_model, tk, xs, bs,horizon, model_defined, epsilon):
    """doc string à ajouter"""
    ts=30 #sampling time
    
    if tk == 0: 
        x0 = xs[tk]
        t0 = tk

        A0_x = 0
        return(prediction(horizon, A0_x, x0, tk), model_defined, (A0_x, t0, x0))
    elif (not model_defined):
        if bs[tk] == 0:
            return(prediction(horizon, current_FLAIR_model[0], current_FLAIR_model[2], ts* tk - current_FLAIR_model[1]), model_defined, current_FLAIR_model)
        else:
            x0, t0 = current_FLAIR_model[2],current_FLAIR_model[1]
            model_defined = True
            x1 = xs[tk]
            t1 = ts*tk
            A0_x = (x1-x0)/(t1-t0 +1e-12)
            A_min_x = (x1-epsilon-x0)/(t1-t0 +1e-12)
            A_max_x = (x1+epsilon-x0)/(t1-t0 +1e-12)
            return(prediction(horizon, A0_x, current_FLAIR_model[2], ts*tk - current_FLAIR_model[1]), model_defined, (A0_x, t0, x0, x1, t1, A_min_x, A_max_x))
    else: 
        if bs[tk] == 0: 
            return(prediction(horizon, current_FLAIR_model[0], current_FLAIR_model[2], ts*tk - current_FLAIR_model[1]), model_defined, current_FLAIR_model)
        else:
            x0, t0 = current_FLAIR_model[2],current_FLAIR_model[1]
            x1, t1 = current_FLAIR_model[3],current_FLAIR_model[4]
            print(t0, t1)
            A0_x, A_min_x, A_max_x = current_FLAIR_model[0], current_FLAIR_model[5], current_FLAIR_model[6]
            x0, t0, x1, t1, A0_x, A_min_x, A_max_x = FLAIR_insert(x0, x1, xs[tk], t0, t1, ts*tk, A0_x, A_min_x, A_max_x, epsilon)
            #print('\n coef directeur: ')
            return(prediction(horizon, A0_x, x0, ts*tk - t0), model_defined, (A0_x, t0, x0, x1, t1, A_min_x, A_max_x)) 

## Symbolic functions


In [5]:
from sys import path

# Control
u = MX.sym("u")
v = MX.sym("v")
b = MX.sym("b")

# State
x = MX.sym("x",nbuf)
y = MX.sym("y",nbuf)
nx = MX.sym("nx",nbuf)

# Matrices
A = np.zeros((nbuf, nbuf))
B = np.zeros((nbuf, 1))
for i in range(nbuf-1):
    A[i,i+1] = 1

B[nbuf-1,0] = 1
A.tolist()
B.tolist()
A = DM(A)
B = DM(B)

# Dynamics
xplus = A@x + B@u
yplus = A@y + B@v
nplus = A@nx + B@b

# Discrete time dynamics function
F = Function('F', [x,u],[xplus])

#Privacity function
O = DM(np.ones((1,nbuf)))
xbar = (O@(x*nx))/((O@nx)+1e-8)
ybar = O@(y*nx)/((O@nx)+1e-8)
P = (O@( sqrt((x-xbar)**2 + (y-ybar)**2)*nx) )/((O@nx)+1e-8)
J = Function('J', [x,y,nx],[xbar, ybar, P],['x','y','nx'],['xbar', 'ybar','P'])

# Test privacy functions
x1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
y1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
nx1 = [1]*nbuf

## MPC function

In [6]:
#Defining mpc function
def solve_mpc_FLAIR(xs,ys,bs,us,horizon,nbuf,util_bound, epsilon):
    current_FLAIR_model_x, current_FLAIR_model_y = (0,0,0), (0,0,0)
    model_defined_x = False
    model_defined_y = False
    model = 0
    Xf_FLAIR, Tx_FLAIR, X_FLAIR, Ax_FLAIR = [], [], [], [] 
    nt = len(xs) 
    x_mpc = []
    y_mpc = []
    U_mpc = []
    V_mpc = []
    barx_mpc = []
    bary_mpc = []
    priv_mpc = []
    util_mpc = []
    time_mpc = []
    X0 = np.array([0]*nbuf).tolist()
    Y0 = np.array([0]*nbuf).tolist()
    N0 = np.array([0]*nbuf).tolist()

    system_opt = pd.DataFrame()
    system_opt['X'] = X0
    system_opt['Y'] = Y0
    system_opt['N'] = N0

    for tk in range(0,nt-horizon):#nt-horizon):
        if bs[tk] == 0:
            U_mpc += [0]
            V_mpc += [0]
            x_mpc += [xs[tk]]
            y_mpc += [ys[tk]]
            system_opt = system_update(system_opt, xs[tk], ys[tk], 0)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt] 
            # Compute utility
            util_mpc += [0]
            time_mpc += [0]
            #system_opt = system_update(system_opt, xs[tk], ys[tk], 0)

        else:
            time_0= time.time()
            X0 = system_opt['X'].tolist()
            Y0 = system_opt['Y'].tolist()
            N0 = system_opt['N'].tolist()

            # Initial conditions
            Jk = 0
            w=[]
            w0 = []
            lbw = []
            ubw = []
            g=[]
            lbg = []
            ubg = []

            # Constraints on input
            U = MX.sym("U",1)
            V = MX.sym("V",1)
            w += [U, V]
            lbw += [-inf]*2
            ubw += [inf]*2
            w0  += [0,util_bound[tk]]
            g   += [U**2 + V**2]
            lbg += [0]
            ubg += [util_bound[tk]**2]

            # Initial conditions
            Xk = MX.sym("Xk",nbuf)
            w += [Xk]
            lbw += X0
            ubw += X0
            w0 += X0
            Yk = MX.sym("Yk",nbuf)
            w += [Yk]
            lbw += Y0
            ubw += Y0
            w0 += Y0
            Nk = MX.sym("Nk",nbuf)
            w += [Nk]
            lbw += N0
            ubw += N0
            w0 += N0

            # Future values of x, y and n
              

            Xf, model_defined_x, current_FLAIR_model_x = predict_xf(current_FLAIR_model_x, tk, xs,bs, horizon, model_defined_x, epsilon)
            if tk!=0:
                Ax_FLAIR.append(current_FLAIR_model_x[0])
                Tx_FLAIR.append(tk*30)
                X_FLAIR.append(current_FLAIR_model_x[3])
            Xf_FLAIR.append(Xf)
            Xf = Xf[::].tolist()

            Yf, model_defined_y, current_FLAIR_model_y = predict_xf(current_FLAIR_model_y, tk, ys,bs, horizon, model_defined_y, epsilon)
            Yf = Yf[::].tolist()
            Nf = np.concatenate((np.array([bs[tk]]), np.ones(horizon - 1))) #comme avec la prédiction de FLAIR on a une prédiction
            Nf = Nf[::].tolist()

            # Apply the first input 
            Xk = F(Xk,Xf[0]+U)
            Yk = F(Yk,Yf[0]+V)
            Nk = F(Nk,Nf[0])
            Jt = J(x=Xk,y=Yk,nx=Nk)
            Jk += Jt['P']
            for k in range(horizon-1):
                # New NLP variable for the control
                Uk = MX.sym('U_' + str(k))
                Vk = MX.sym('V_' + str(k))
                w   += [Uk, Vk]
                lbw += [-inf]*2
                ubw += [inf]*2
                w0 += [0]*2

                g   += [Uk**2 + Vk**2]
                lbg += [0]
                ubg += [util_bound[tk+k+1]**2]

                Xk = F(Xk,Xf[k+1]+Uk)
                Yk = F(Yk,Yf[k+1]+Vk)
                Nk = F(Nk,Nf[k+1])
                Jt = J(x=Xk,y=Yk,nx=Nk)
                Jk += Jt['P']


            prob = {'f': -Jk, 'x': vertcat(*w), 'g': vertcat(*g)}
            solver = nlpsol('solver', 'ipopt', prob);
            sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
            w_opt = sol['x'].full().flatten()
            J_opt = -sol['f']

            # Plot the solution
            u_opt = w_opt[0]
            v_opt = w_opt[1]
            U_mpc += [u_opt]
            V_mpc += [v_opt]
            x_mpc += [Xf[0] + u_opt]
            y_mpc += [Yf[0] + v_opt]
            system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt] 
            # Compute utility
            util_mpc += [utility(u_opt,v_opt)]
            time_mpc += [time.time()-time_0]
            #system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)
   
    clear_output(wait=False)

    data_mpc = pd.DataFrame()
    data_mpc['x'] = x_mpc
    data_mpc['y'] = y_mpc
    data_mpc['u_enu']= us[:len(x_mpc)]
    data_mpc['U'] = U_mpc
    data_mpc['V'] = V_mpc
    data_mpc['barx'] = barx_mpc
    data_mpc['bary'] = bary_mpc
    data_mpc['priv'] = priv_mpc
    data_mpc['util'] = util_mpc
    data_mpc['time'] = time_mpc
    
    
    #return data_mpc, Ax_FLAIR, Tx_FLAIR, X_FLAIR, Xf_FLAIR
    return data_mpc#U_mpc, V_mpc, x_mpc, y_mpc, barx_mpc, bary_mpc, priv_mpc, util_mpc 

## Main function

In [7]:
def func_run(user_n,horizons,GeoI_filename):
    user= str(user_n) # Examples: abboip oilrag 51 90 14
    #Name your files
    local_url="datasets/"+dataset+"-tree/"+user+".csv"
    folder_sol='FLAIR_20000'
    sol_file='solutions/'+dataset+'_10s/'+folder_sol+'/sol_' +str(user_n)
    file_GeoI='solutions/'+dataset+'_10s/'+GeoI_filename+'/sol_' +str(user_n)
    exists = os.path.isfile(file_GeoI+'/Data_GeoI.csv')
    if exists:
        data = pd.read_csv(local_url,names=["latitude","longitude","timestamp"])
        print("uploaded from local file")
    else:
        return #if user doesn't exist return out of the function in order to get another user


    if not os.path.exists(sol_file):
        os.makedirs(sol_file)
        if not os.path.isfile('solutions/'+dataset+'_10s/'+folder_sol+'/info.txt'):
            text_file = open('solutions/'+dataset+'_10s/'+folder_sol+'/info.txt', "w")
            ox = text_file.write('GeoI folder used : '+GeoI_filename)
            text_file.close()
        
        
    Data_GeoI=pd.read_csv(file_GeoI+'/Data_GeoI.csv')    
    Data_real=pd.read_csv(file_GeoI+'/Data_real.csv')
    xs=Data_real['x']
    ys=Data_real['y'] 
    bs=Data_real['b']
    us=Data_real['u_enu']
    util=Data_GeoI['util'] 
    util2=Data_GeoI.loc[Data_GeoI['util']>0].mean()
    
    erreur = 5*1e-2 #à préciser
    start_time = time.monotonic()
    Data_MPC = {} # Empty dictionary
    for h in horizons:
        data_mpc = solve_mpc_FLAIR(xs,ys,bs,us,h,nbuf,util,erreur)
        Data_MPC[h] = data_mpc
        end_time = time.monotonic()
        print('user',user,' horizon ',h,' runtime ',timedelta(seconds=end_time - start_time), ' total time ',timedelta(seconds=end_time - time_0))
        Data_MPC[h].to_csv(sol_file+'\\save_data_mpc_horizon_' + str(h) + '.csv')


    end_time = time.monotonic()
    print(timedelta(seconds=end_time - start_time))


    # Save data MPC
    pickle.dump( horizons, open( sol_file+"/Horizons_MPC.pkl", "wb" ) )
    pd.DataFrame(horizons).to_csv(sol_file+"/horizons.csv") 
    pickle.dump( Data_MPC, open( sol_file+"/Data_MPC.pkl", "wb" ) )
    
   

## Run MPC

In [ ]:
dataset="privamov" # OPTIONS ARE: cabspotting privamov
#horizon 7->3min
hor=[1,2,3,5,8,13,16]#[1,2,3,4,5,6,7]#,8,9,10]
time_0 = time.monotonic()
for user in range(1,8):
    func_run(user,hor,'GeoI_20000')

user 1  horizon  13  runtime  16:26:22.266000  total time  16:26:22.391000
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       32
Number of nonzeros in Lagrangian Hessian.............:      528

Total number of variables............................:       32
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       16
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:       16
        inequality constraints with only upper bounds:        0

iter    obje

  77 -2.2103169e+002 0.00e+000 9.48e-002  -2.5 1.30e-001  -0.1 1.00e+000 1.00e+000h  1
  78 -2.2112380e+002 0.00e+000 9.43e-002  -2.5 3.89e-001  -0.6 1.00e+000 1.00e+000f  1
  79 -2.2115790e+002 0.00e+000 9.42e-002  -2.5 1.45e-001  -0.2 1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 -2.2126012e+002 0.00e+000 9.36e-002  -2.5 4.34e-001  -0.7 1.00e+000 1.00e+000f  1
  81 -2.2129791e+002 0.00e+000 9.35e-002  -2.5 1.62e-001  -0.2 1.00e+000 1.00e+000h  1
  82 -2.2141118e+002 0.00e+000 9.29e-002  -2.5 4.84e-001  -0.7 1.00e+000 1.00e+000f  1
  83 -2.2145301e+002 0.00e+000 9.27e-002  -2.5 1.81e-001  -0.3 1.00e+000 1.00e+000h  1
  84 -2.2157835e+002 0.00e+000 9.20e-002  -2.5 5.40e-001  -0.8 1.00e+000 1.00e+000f  1
  85 -2.2162456e+002 0.00e+000 9.18e-002  -2.5 2.02e-001  -0.3 1.00e+000 1.00e+000h  1
  86 -2.2176304e+002 0.00e+000 9.11e-002  -2.5 6.01e-001  -0.8 1.00e+000 1.00e+000f  1
  87 -2.2181402e+002 0.00e+000 9.09e-002  -2.5 2.

 166 -2.2624082e+002 0.00e+000 7.38e-002  -2.5 1.06e-001  -0.2 1.00e+000 1.00e+000h  1
 167 -2.2630126e+002 0.00e+000 7.36e-002  -2.5 3.17e-001  -0.6 1.00e+000 1.00e+000f  1
 168 -2.2632378e+002 0.00e+000 7.36e-002  -2.5 1.19e-001  -0.2 1.00e+000 1.00e+000h  1
 169 -2.2639133e+002 0.00e+000 7.34e-002  -2.5 3.55e-001  -0.7 1.00e+000 1.00e+000f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170 -2.2641648e+002 0.00e+000 7.33e-002  -2.5 1.33e-001  -0.3 1.00e+000 1.00e+000h  1
 171 -2.2649191e+002 0.00e+000 7.31e-002  -2.5 3.97e-001  -0.7 1.00e+000 1.00e+000f  1
 172 -2.2651998e+002 0.00e+000 7.30e-002  -2.5 1.49e-001  -0.3 1.00e+000 1.00e+000h  1
 173 -2.2660415e+002 0.00e+000 7.27e-002  -2.5 4.45e-001  -0.8 1.00e+000 1.00e+000f  1
 174 -2.2663545e+002 0.00e+000 7.26e-002  -2.5 1.67e-001  -0.4 1.00e+000 1.00e+000h  1
 175 -2.2672928e+002 0.00e+000 7.24e-002  -2.5 4.98e-001  -0.8 1.00e+000 1.00e+000f  1
 176 -2.2676414e+002 0.00e+000 7.23e-002  -2.5 1.

 257 -2.2968172e+002 0.00e+000 6.45e-002  -2.5 1.08e-001  -0.2 1.00e+000 1.00e+000h  1
 258 -2.2973088e+002 0.00e+000 6.43e-002  -2.5 3.25e-001  -0.7 1.00e+000 1.00e+000f  1
 259 -2.2974922e+002 0.00e+000 6.43e-002  -2.5 1.22e-001  -0.3 1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 260 -2.2980422e+002 0.00e+000 6.42e-002  -2.5 3.64e-001  -0.8 1.00e+000 1.00e+000f  1
 261 -2.2982472e+002 0.00e+000 6.41e-002  -2.5 1.36e-001  -0.3 1.00e+000 1.00e+000h  1
 262 -2.2988620e+002 0.00e+000 6.39e-002  -2.5 4.08e-001  -0.8 1.00e+000 1.00e+000f  1
 263 -2.2990910e+002 0.00e+000 6.39e-002  -2.5 1.53e-001  -0.4 1.00e+000 1.00e+000h  1
 264 -2.2997778e+002 0.00e+000 6.37e-002  -2.5 4.58e-001  -0.9 1.00e+000 1.00e+000f  1
 265 -2.3000334e+002 0.00e+000 6.37e-002  -2.5 1.72e-001  -0.4 1.00e+000 1.00e+000h  1
 266 -2.3001294e+002 0.00e+000 6.36e-002  -2.5 6.43e-002  -0.0 1.00e+000 1.00e+000h  1
 267 -2.3004172e+002 0.00e+000 6.36e-002  -2.5 1.

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 350 -2.3225808e+002 0.00e+000 5.84e-002  -2.5 1.30e-001  -0.3 1.00e+000 1.00e+000h  1
 351 -2.3231137e+002 0.00e+000 5.83e-002  -2.5 3.89e-001  -0.8 1.00e+000 1.00e+000f  1
 352 -2.3233125e+002 0.00e+000 5.83e-002  -2.5 1.46e-001  -0.4 1.00e+000 1.00e+000h  1
 353 -2.3233871e+002 0.00e+000 5.83e-002  -2.5 5.46e-002   0.0 1.00e+000 1.00e+000h  1
 354 -2.3236109e+002 0.00e+000 5.82e-002  -2.5 1.64e-001  -0.4 1.00e+000 1.00e+000f  1
 355 -2.3236947e+002 0.00e+000 5.82e-002  -2.5 6.14e-002  -0.0 1.00e+000 1.00e+000h  1
 356 -2.3239458e+002 0.00e+000 5.81e-002  -2.5 1.84e-001  -0.5 1.00e+000 1.00e+000f  1
 357 -2.3240398e+002 0.00e+000 5.81e-002  -2.5 6.90e-002  -0.1 1.00e+000 1.00e+000h  1
 358 -2.3243216e+002 0.00e+000 5.81e-002  -2.5 2.07e-001  -0.6 1.00e+000 1.00e+000f  1
 359 -2.3244270e+002 0.00e+000 5.80e-002  -2.5 7.75e-002  -0.1 1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d|

...

  99 -4.7921808e+003 6.55e-010 2.76e-006  -8.6 1.49e-007    -  1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100 -4.7921808e+003 6.46e-010 2.65e-006  -8.6 1.45e-007    -  1.00e+000 1.00e+000h  1
 101 -4.7921808e+003 6.38e-010 2.55e-006  -8.6 1.41e-007    -  1.00e+000 1.00e+000h  1
 102 -4.7921808e+003 6.30e-010 2.45e-006  -8.6 1.38e-007    -  1.00e+000 1.00e+000h  1
 103 -4.7921808e+003 6.22e-010 2.36e-006  -8.6 1.34e-007    -  1.00e+000 1.00e+000h  1
 104 -4.7921808e+003 6.15e-010 2.28e-006  -8.6 1.31e-007    -  1.00e+000 1.00e+000h  1
 105 -4.7921808e+003 6.07e-010 2.20e-006  -8.6 1.28e-007    -  1.00e+000 1.00e+000h  1
 106 -4.7921808e+003 6.00e-010 2.12e-006  -8.6 1.25e-007    -  1.00e+000 1.00e+000h  1
 107 -4.7921808e+003 5.93e-010 2.05e-006  -8.6 1.22e-007    -  1.00e+000 1.00e+000h  1
 108 -4.7921808e+003 5.87e-010 1.98e-006  -8.6 1.19e-007    -  1.00e+000 1.00e+000h  1
 109 -4.7921808e+003 5.80e-010 1.92e-006  -8.6 1.

   9 -4.6849334e+003 9.45e-004 1.16e-001  -2.5 7.77e-001  -0.8 1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -4.6853920e+003 2.36e-004 1.20e-001  -2.5 2.40e+000  -1.3 1.00e+000 1.00e+000h  1
  11 -4.6867484e+003 5.91e-005 1.31e-001  -2.5 7.88e+000  -1.8 1.00e+000 1.00e+000h  1
  12 -4.6917170e+003 8.05e+002 1.62e-001  -2.5 3.05e+001  -2.3 1.00e+000 1.00e+000h  1
  13 -4.7210173e+003 2.83e+004 1.54e-001  -2.5 1.49e+002  -2.7 1.00e+000 1.00e+000f  1
  14 -4.7639997e+003 1.25e+005 2.44e-002  -2.5 3.53e+002  -3.2 1.00e+000 1.00e+000h  1
  15 -4.7627786e+003 1.18e+005 2.31e-002  -2.5 1.23e+003    -  3.85e-001 5.89e-002h  1
  16 -4.7581703e+003 9.29e+004 1.94e-002  -2.5 1.56e+002    -  1.00e+000 2.24e-001h  1
  17 -4.7422928e+003 2.05e+004 1.33e-001  -2.5 1.64e+002    -  3.41e-001 1.00e+000h  1
  18 -4.7316223e+003 3.29e+003 2.36e-002  -2.5 4.21e+001  -3.7 1.00e+000 1.00e+000h  1
  19 -4.7435352e+003 1.83e+002 1.36e-001  -2.5 5.

  98 -4.7910470e+003 7.59e-010 4.04e-006  -8.6 1.89e-007    -  1.00e+000 1.00e+000h  1
  99 -4.7910470e+003 7.48e-010 3.87e-006  -8.6 1.83e-007    -  1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100 -4.7910470e+003 7.37e-010 3.71e-006  -8.6 1.78e-007    -  1.00e+000 1.00e+000h  1
 101 -4.7910470e+003 7.27e-010 3.56e-006  -8.6 1.73e-007    -  1.00e+000 1.00e+000h  1
 102 -4.7910470e+003 7.17e-010 3.42e-006  -8.6 1.68e-007    -  1.00e+000 1.00e+000h  1
 103 -4.7910470e+003 7.08e-010 3.28e-006  -8.6 1.64e-007    -  1.00e+000 1.00e+000h  1
 104 -4.7910470e+003 6.99e-010 3.16e-006  -8.6 1.60e-007    -  1.00e+000 1.00e+000h  1
 105 -4.7910470e+003 6.90e-010 3.04e-006  -8.6 1.56e-007    -  1.00e+000 1.00e+000h  1
 106 -4.7910470e+003 6.81e-010 2.92e-006  -8.6 1.52e-007    -  1.00e+000 1.00e+000h  1
 107 -4.7910470e+003 6.73e-010 2.82e-006  -8.6 1.48e-007    -  1.00e+000 1.00e+000h  1
 108 -4.7910470e+003 6.65e-010 2.72e-006  -8.6 1.

   1 -4.6945777e+003 1.58e+002 3.01e+002  -1.0 1.39e+001  -2.0 9.90e-001 9.90e-001f  1
   2 -4.6885251e+003 1.95e+001 3.02e+002  -1.0 6.90e+000   2.0 3.40e-001 1.00e+000h  1
   3 -4.6857303e+003 4.86e+000 7.53e+001  -1.0 1.90e+001    -  1.01e-002 1.00e+000f  1
   4 -4.6844881e+003 1.22e+000 5.02e+001  -1.0 1.52e+000   1.6 9.91e-001 1.00e+000h  1
   5 -4.6841089e+003 3.04e-001 1.37e+001  -1.0 5.34e-001   1.1 1.00e+000 1.00e+000h  1
   6 -4.6840447e+003 7.60e-002 3.45e+000  -1.0 2.67e-001   0.6 1.00e+000 1.00e+000h  1
   7 -4.6840503e+003 1.90e-002 9.30e-001  -1.0 1.33e-001   0.1 1.00e+000 1.00e+000h  1
   8 -4.6840978e+003 4.75e-003 2.26e-001  -1.7 2.64e-001  -0.3 1.00e+000 1.00e+000h  1
   9 -4.6842663e+003 1.19e-003 1.20e-001  -1.7 7.99e-001  -0.8 1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -4.6847550e+003 2.97e-004 1.23e-001  -1.7 2.46e+000  -1.3 1.00e+000 1.00e+000h  1
  11 -4.6861531e+003 7.42e-005 1.32e-001  -1.7 7.

  92 -4.7846666e+003 8.93e-010 6.16e-006  -8.6 2.44e-007    -  1.00e+000 1.00e+000h  1
  93 -4.7846666e+003 8.78e-010 5.87e-006  -8.6 2.36e-007    -  1.00e+000 1.00e+000h  1
  94 -4.7846666e+003 8.64e-010 5.59e-006  -8.6 2.29e-007    -  1.00e+000 1.00e+000h  1
  95 -4.7846666e+003 8.51e-010 5.33e-006  -8.6 2.22e-007    -  1.00e+000 1.00e+000h  1
  96 -4.7846666e+003 8.38e-010 5.09e-006  -8.6 2.15e-007    -  1.00e+000 1.00e+000h  1
  97 -4.7846666e+003 8.25e-010 4.87e-006  -8.6 2.09e-007    -  1.00e+000 1.00e+000h  1
  98 -4.7846666e+003 8.13e-010 4.66e-006  -8.6 2.03e-007    -  1.00e+000 1.00e+000h  1
  99 -4.7846666e+003 8.02e-010 4.46e-006  -8.6 1.97e-007    -  1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100 -4.7846666e+003 7.91e-010 4.28e-006  -8.6 1.91e-007    -  1.00e+000 1.00e+000h  1
 101 -4.7846666e+003 7.80e-010 4.11e-006  -8.6 1.86e-007    -  1.00e+000 1.00e+000h  1
 102 -4.7846666e+003 7.70e-010 3.94e-006  -8.6 1.